In [120]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb

In [121]:
df = pd.read_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\train_set.csv")
df.head()

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,...,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,...,0.033431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,...,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,...,0.039363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,...,0.069242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [122]:
X= df.iloc[:, 1:78]  # All columns except the last one
y = df.iloc[:, 78]   # Only the last column

In [123]:
#selector = VarianceThreshold(0.5)
#X = selector.fit_transform(X)
df = df.dropna(axis=0)

In [124]:
#scaled_data = SimpleImputer(strategy='mean')
#X = scaled_data.fit_transform(X)
#X = pd.DataFrame(X)

#Normalize
# Sample data
data = df

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the data
df = scaler.fit_transform(data)

In [125]:
#standardizer = StandardScaler()
#standardizedX = standardizer.fit_transform(X)
#X = pd.DataFrame(standardizedX)

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=100, stratify=y)

In [127]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [128]:
params = {
    'objective': 'binary',          # Change to 'multiclass' if multiclass classification
    'boosting_type': 'gbdt',        # Gradient Boosting Decision Tree
    'metric': 'binary_logloss',     # Change to 'multi_logloss' if multiclass classification
    'learning_rate': 0.05,
    'num_leaves': 21,
    'max_depth': 12,
    'seed': 42
}

In [129]:
lgbm_model = lgb.train(params, train_data, valid_sets=[test_data])

[LightGBM] [Info] Number of positive: 584, number of negative: 220925
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.077002 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17722
[LightGBM] [Info] Number of data points in the train set: 221509, number of used features: 77
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.002636 -> initscore=-5.935678
[LightGBM] [Info] Start training from score -5.935678


In [130]:
# Make predictions
y_pred = lgbm_model.predict(X_test, num_iteration=lgbm_model.best_iteration)
y_pred_class = [1 if pred > 0.5 else 0 for pred in y_pred]  # Adjust threshold if necessary

In [131]:
accuracy = accuracy_score(y_test, y_pred_class)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred_class))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_class))

Accuracy: 0.9967090561898184
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     24548
           1       0.23      0.11      0.15        65

    accuracy                           1.00     24613
   macro avg       0.62      0.55      0.57     24613
weighted avg       1.00      1.00      1.00     24613

Confusion Matrix:
 [[24525    23]
 [   58     7]]


In [132]:
full_data = lgb.Dataset(X, label=y)  # Create a dataset with all data

In [133]:
# Set the number of boosting rounds for the final model
num_boost_round = lgbm_model.best_iteration if lgbm_model.best_iteration > 0 else 100  # Use 100 as a default if 0

# Train the LightGBM model on the full data
lgbm_final_model = lgb.train(params, full_data, num_boost_round=num_boost_round)


[LightGBM] [Info] Number of positive: 649, number of negative: 245473
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.068698 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17725
[LightGBM] [Info] Number of data points in the train set: 246122, number of used features: 77
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.002637 -> initscore=-5.935510
[LightGBM] [Info] Start training from score -5.935510


In [134]:
test_data = pd.read_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\test_set.csv")
recordId = test_data['RecordId']
test_data = test_data.drop(columns = 'RecordId')

In [135]:
#test_data = selector.transform(test_data)
test_data = SimpleImputer(strategy='mean').fit_transform(test_data)
#minMaxTest = minmaxer.fit_transform(test_data)
#test_data = standardizer.fit_transform(test_data)
#test_data = pd.DataFrame(test_data)
#test_data = pca.transform(test_data)

#Normalize
# Sample data
#data = test_data

# Initialize the scaler
#scaler = MinMaxScaler()

# Fit and transform the data
#test_data = scaler.fit_transform(data)

In [136]:
prediction = lgbm_final_model.predict(test_data)  # Predicts probabilities directly
#prediction = prediction[:, 1]
# Convert predictions to DataFrame format
finaldf = pd.concat([recordId, pd.DataFrame(prediction, columns=['Y'])], axis=1)
finaldf.to_csv("C:\\Users\\Ali\\DataScience_Projects\\Kaggle_Competition\\lightgbm_test_results.csv", index=False)
